In [1]:
import json
import pandas as pd
import numpy as np



In [2]:

!git clone https://huggingface.co/datasets/newbienewbie/handwriting_edge_case_in_one_stroke



Cloning into 'handwriting_edge_case_in_one_stroke'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 44 (delta 8), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (44/44), 10.94 KiB | 260.00 KiB/s, done.


In [4]:
file_path = "/content/handwriting_edge_case_in_one_stroke/data/test-00000-of-00001.parquet"
testData = pd.read_parquet(file_path)

file_path = "/content/handwriting_edge_case_in_one_stroke/data/train-00000-of-00001.parquet"
trainData = pd.read_parquet(file_path)

file_path = "/content/handwriting_edge_case_in_one_stroke/data/validation-00000-of-00001.parquet"
validationData = pd.read_parquet(file_path)


# Inspect the strokes column
# print(testData)

In [5]:

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

# Extracting x, y, time, and annotations
xCoordinates = [np.array(item, dtype=np.float32) for item in trainData['xCoordinates'].tolist()]
yCoordinates = [np.array(item, dtype=np.float32) for item in trainData['yCoordinates'].tolist()]
# timeStamps = [np.array(item, dtype=np.float32) for item in trainData['timeStamps'].tolist()]
annotations = [str(item) for item in trainData['sampleTag']]



# Normalize coordinates and timestamps
scaler = MinMaxScaler()

xCoordinates = [scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in xCoordinates]
yCoordinates = [scaler.fit_transform(y.reshape(-1, 1)).flatten() for y in yCoordinates]
# timeStamps = [scaler.fit_transform(t.reshape(-1, 1)).flatten() for t in timeStamps]

# Combine and pad sequences
combinedData = [np.column_stack((x, y)) for x, y in zip(xCoordinates, yCoordinates)]


xCoordinates_v = [np.array(item, dtype=np.float32) for item in validationData['xCoordinates'].tolist()]
yCoordinates_v = [np.array(item, dtype=np.float32) for item in validationData['yCoordinates'].tolist()]
timeStamps_v = [np.array(item, dtype=np.float32) for item in validationData['timeStamps'].tolist()]
annotations_v = [str(item) for item in validationData['sampleTag']]



# Normalize coordinates and timestamps
scaler = MinMaxScaler()

xCoordinates_v = [scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in xCoordinates_v]
yCoordinates_v = [scaler.fit_transform(y.reshape(-1, 1)).flatten() for y in yCoordinates_v]
timeStamps_v = [scaler.fit_transform(t.reshape(-1, 1)).flatten() for t in timeStamps_v]

# Combine and pad sequences
combinedData_v = [np.column_stack((x, y)) for x, y in zip(xCoordinates_v, yCoordinates_v)]
random.shuffle(combinedData_v)
# print(combinedData)
print(xCoordinates[0])

[0.7111112  0.61111116 0.5555556  0.48888898 0.41111112 0.3777778
 0.32222223 0.22222221 0.17777777 0.13333333 0.08888888 0.05555558
 0.03333336 0.01111114 0.         0.         0.         0.
 0.         0.         0.02222222 0.04444444 0.07777774 0.10000002
 0.13333333 0.16666675 0.20000005 0.32222223 0.51111114 0.61111116
 0.7222222  0.88888896 1.        ]


In [6]:

#all
lengths = [len(seq) for seq in combinedData]
max_len = int(np.percentile(lengths, 95))  # Adjust according to your needs
print(max_len)
padded_data_all = pad_sequences(combinedData, maxlen=max_len, padding='post', dtype='float32')


# One-hot encode annotations
label_binarizer = LabelBinarizer()

all_annotations =  annotations # Combine all annotations
label_binarizer = LabelBinarizer()
label_binarizer.fit(all_annotations)
labels_all = label_binarizer.transform(annotations)

# print(labels_60)

36


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(padded_data_all.shape[1], padded_data_all.shape[2])))
model.add(LSTM(50))
model.add(Dense(labels_all.shape[1], activation='softmax'))  # Number of classes

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=2,              # Number of epochs to wait before stopping
    restore_best_weights=True  # Restore weights of the best epoch
)

# Train the model
model.fit(padded_data_all, labels_all, epochs=20, batch_size=1, callbacks=[early_stopping])

print(labels_all)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.4208 - loss: 1.5753
Epoch 2/20
 12/858 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8595 - loss: 0.8326

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.6710 - loss: 0.8913
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7536 - loss: 0.6718
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.7973 - loss: 0.5578
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.7530 - loss: 0.6779
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.8106 - loss: 0.4918
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8029 - loss: 0.4844
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8364 - loss: 0.4351
Epoch 9/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.8527 - loss: 0.3828
Epoch 10/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8611 - loss: 0.3508
Epoch 11/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8703 - loss: 0.3334
Epoch 12/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.8763 - loss: 0.3145
Epoch 13/20
858/858 ━━━━━━━━━━━━━━━━━━━━

In [8]:

# Testing Extracting x, y, time, and annotations

# Extracting x, y, time, and annotations
xCoordinates_t = [np.array(item, dtype=np.float32) for item in testData['xCoordinates'].tolist()]
yCoordinates_t = [np.array(item, dtype=np.float32) for item in testData['yCoordinates'].tolist()]
timeStamps_t = [np.array(item, dtype=np.float32) for item in testData['timeStamps'].tolist()]
annotations_t = [str(item) for item in testData['sampleTag']]



# Normalize coordinates and timestamps
scaler = MinMaxScaler()

xCoordinates_t = [scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in xCoordinates_t]
yCoordinates_t = [scaler.fit_transform(y.reshape(-1, 1)).flatten() for y in yCoordinates_t]
timeStamps_t = [scaler.fit_transform(t.reshape(-1, 1)).flatten() for t in timeStamps_t]

# Combine and pad sequences
combinedData_t = [np.column_stack((x, y)) for x, y in zip(xCoordinates_t, yCoordinates_t)]
# print(combinedData)

In [11]:
lengths = [len(seq) for seq in combinedData_v]
max_len = int(np.percentile(lengths, 90))  # Adjust according to your needs
print(max_len)
padded_data_v = pad_sequences(combinedData_v, maxlen=max_len, padding='post', dtype='float32')

# One-hot encode annotations
label_binarizer = LabelBinarizer()
labels_v = label_binarizer.fit_transform(annotations_v)


eval = model.evaluate(padded_data_v, labels_v)
# import matplotlib.pyplot as plt

# # Log results for comparison
# experiment_results = {"loss": eval[0], "accuracy": eval[1]}

# # Example visualization
# plt.bar(experiment_results.keys(), experiment_results.values())
# plt.title("Model Evaluation Metrics")
# plt.show()

37


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 8), output.shape=(None, 7)

In [9]:
def predict_sample(input_data):
    # Preprocess the input
    padded_input = pad_sequences([input_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model.predict(padded_input)
    predicted_class = np.argmax(prediction, axis=1)


    # Map to label
    label = label_binarizer.inverse_transform(prediction)
    return label

In [10]:
original_labels = label_binarizer.classes_
print("Original Labels:", original_labels)
for i, test_data in enumerate(combinedData_t):
    lengths = [len(seq) for seq in combinedData_t]
    max_len = int(np.percentile(lengths, 95))  # Adjust according to your needs

    test_data_padded = pad_sequences([test_data], maxlen=max_len, padding='post', dtype='float32')

    # Predict
    prediction = model.predict(test_data_padded)


    # Interpret
    predicted_label = label_binarizer.inverse_transform(prediction)


    print(f"Sample {i}: Predicted label: {predicted_label}")
    print(f"Sample {i}: Predicted Value: {np.max(prediction)}")
    print(f"Sample {i}: Actual label: {annotations_t[i]}")


#Save the model in the SavedModel format (default)
model.save('model_all_0109_2strokes_default.keras')

# Save the model in HDF5 format
model.save('model_all_0109_2strokes.h5')

Original Labels: ['(' ')' '7' 'C' 'bs' 'h' 'v']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Sample 0: Predicted label: ['v']
Sample 0: Predicted Value: 0.9870332479476929
Sample 0: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sample 1: Predicted label: ['v']
Sample 1: Predicted Value: 0.9643728733062744
Sample 1: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Sample 2: Predicted label: ['v']
Sample 2: Predicted Value: 0.9412593841552734
Sample 2: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Sample 3: Predicted label: ['v']
Sample 3: Predicted Value: 0.841128408908844
Sample 3: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sample 4: Predicted label: ['v']
Sample 4: Predicted Value: 0.9348499774932861
Sample 4: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Sample 5: Predicted label: ['v']
Sample 5: Predicted Value: 0.9754005074501038
Sample 5: Actual label: v
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Sample 6: Predicted label: ['v']
Sample 6: Predicted Val

In [21]:
import tensorflow as tf
import os
print(os.listdir('/content/'))
# Convert the SavedModel to TFLite
# Load the Keras model
model_keras = tf.keras.models.load_model('/content/model_all_0109_2strokes.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model_keras)

# Use only supported ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Disable the conversion of tensor list ops
converter._experimental_lower_tensor_list_ops = False


tflite_model = converter.convert()

# Save the converted model
with open('/content/model_all_010.tflite', 'wb') as f:
    f.write(tflite_model)

['.config', 'handwriting_strokes_2strokes', 'model_all_0109_2strokes.h5', '.ipynb_checkpoints', 'model_all_0109_2strokes_default.keras', 'sample_data']
Saved artifact at '/tmp/tmps7acrdjn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 43, 2), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 16), dtype=tf.float32, name=None)
Captures:
  133996496879312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133996496878432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133996496384800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133996496387088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133996496373360: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [38]:
import tensorflow as tf
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/content/model_all_010.tflite')

# Allocate tensors (required before inference)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print the input and output shapes
print("Input details:", input_details)
print("Output details:", output_details)

# Example input data (use actual input shape and data for your model)
input_shape = input_details[0]['shape']
dummy_input = np.zeros(input_shape, dtype=np.float32)


for i, test_data in enumerate(combinedData_t):
    lengths = [len(seq) for seq in combinedData_t]
    max_len = int(np.percentile(lengths, 95))  # Adjust according to your needs
    # print(max_len)
    max_len = 43
    test_data_padded = pad_sequences([test_data], maxlen=max_len, padding='post', dtype='float32')

    print(test_data_padded.shape)
    # Perform inference
    interpreter.set_tensor(input_details[0]['index'], test_data_padded)
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(output_data.shape)
    # Interpret
    predicted_label = label_binarizer.inverse_transform(output_data)
    # print("Output shape:", output_data.shape)
    print(f"Prediction {i}: {predicted_label}")
    print(f"Sample {i}: Actual label: {annotations_t[i]}")
    print(" ")


Input details: [{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([ 1, 43,  2], dtype=int32), 'shape_signature': array([-1, 43,  2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 64, 'shape': array([ 1, 16], dtype=int32), 'shape_signature': array([-1, 16], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
(1, 43, 2)
(1, 16)
Prediction 0: ['5_0']
Sample 0: Actual label: 5_0
 
(1, 43, 2)
(1, 16)
Prediction 1: ['5_1']
Sample 1: Actual label: 5_1
 
(1, 43, 2)
(1, 16)
Prediction 2: ['v']
Sample 2: Actual label: v
 
(1, 43, 2)
(1, 16)
Prediction 3

In [24]:
print(annotations)

['0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '5_1', '8_1', '9_1', '10_1', '11_0', '12_1', '14_1', '15_1', '16_1', '0_0', '1_1', '2_1', '5_0', '

In [14]:

# Function to calculate the curvature between 3 consecutive points
def calculate_curvature(x1, y1, x2, y2, x3, y3):
    # Calculate the distances between points
    d1 = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)  # Distance between P1 and P2
    d2 = np.sqrt((x3 - x2)**2 + (y3 - y2)**2)  # Distance between P2 and P3
    d3 = np.sqrt((x3 - x1)**2 + (y3 - y1)**2)  # Distance between P1 and P3

    # Calculate the curvature using the determinant-based formula
    numerator = 2 * abs((x1 - x2) * (y2 - y3) - (x2 - x3) * (y1 - y2))
    denominator = d1 * d2 * d3

    # Return curvature
    return numerator / denominator if denominator != 0 else 0

# Example data
xCoordinates = ["53.5", "46.0", "46.0", "46.0", "46.0", "46.0", "45.5", "44.0"]
yCoordinates = ["22.5", "92.0", "96.0", "99.0", "101.0", "102.5", "101.5", "97.0"]

# Convert to float
x_coordinates = list(map(float, xCoordinates))
y_coordinates = list(map(float, yCoordinates))

# Calculate curvature for each set of 3 consecutive points
curvatures = []
for i in range(len(x_coordinates) - 2):
    curvature = calculate_curvature(x_coordinates[i], y_coordinates[i],
                                    x_coordinates[i+1], y_coordinates[i+1],
                                    x_coordinates[i+2], y_coordinates[i+2])
    curvatures.append(curvature)

# Display the curvature values
print("Curvature values for each set of 3 points:", curvatures)

Curvature values for each set of 3 points: [0.002904394797674704, 0.0, 0.0, 0.0, 1.2649110640673515, 0.04832976746641415]
